In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

In [ ]:

def forward_stepwise_regression(X, y, cv=5):
    # Inizializza una lista per tenere traccia delle variabili selezionate
    selected_features = []
    
    # Inizializza il modello di regressione lineare
    model = LinearRegression()
    
    # Numero di variabili
    n_features = X.shape[1]
    
    # Itera su tutte le variabili
    for i in range(n_features):
        best_score = -np.inf
        best_feature = None
        
        # Itera su tutte le variabili non ancora selezionate
        for feature in X.columns:
            if feature not in selected_features:
                # Aggiungi la variabile corrente alle variabili selezionate
                selected_features.append(feature)
                
                # Addestra il modello utilizzando le variabili selezionate
                model.fit(X[selected_features], y)
                
                # Valuta il modello utilizzando la cross-validation
                scores = cross_val_score(model, X[selected_features], y, cv=cv, scoring='neg_mean_squared_error')
                mean_score = scores.mean()
                
                # Se il punteggio medio è migliore del miglior punteggio precedente, aggiorna il miglior punteggio e la miglior variabile
                if mean_score > best_score:
                    best_score = mean_score
                    best_feature = feature
                
                # Rimuovi la variabile corrente dalle variabili selezionate per testare la prossima variabile
                selected_features.remove(feature)
        
        # Aggiungi la miglior variabile trovata alla lista delle variabili selezionate
        selected_features.append(best_feature)
        print(f"Step {i+1}: Added feature '{best_feature}', Mean Negative Mean Squared Error: {best_score:.2f}")
    
    return selected_features

In [ ]:

df = pd.read_csv("Dataset/universal_top_spotify_songs.csv")
df.head()

In [1]:
numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
X = df[numeric_columns].drop(['daily_rank'], axis=1)  

categorical_columns = ['artists', 'country']
for col in categorical_columns:
    X = pd.concat([X, pd.get_dummies(df[col], prefix=col)], axis=1)

y = df['daily_rank']

selected_features = forward_stepwise_regression(X, y)

print("Selected features:", selected_features)


KeyboardInterrupt: 